In [1]:
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv2D, MaxPool2D, Flatten
from keras import optimizers
from keras import losses
from keras.utils import plot_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import time, os, fnmatch, shutil

In [3]:
cmmd_dir = "F:/uni/UT/Y-Y/"

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        shear_range=0.2,
        zoom_range=0.2,
        vertical_flip=True,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(
        cmmd_dir+"cmmd_data/PNG/TRAIN/",
        target_size=(227, 227),
        color_mode="grayscale",
        batch_size=8,
        shuffle=True,
        class_mode='binary',
        seed=14)
validation_generator = test_datagen.flow_from_directory(
        cmmd_dir+"cmmd_data/PNG/VAL/",
        target_size=(227, 227),
        color_mode="grayscale",
        batch_size=8,
        shuffle=True,
        class_mode='binary',
        seed=14)

Found 2402 images belonging to 2 classes.
Found 602 images belonging to 2 classes.


# Modified AlexNet implementation

In [5]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, 
                        kernel_size=(11,11), 
                        strides=(4,4), 
                        activation='relu', 
                        input_shape=(227,227,1)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), 
                           strides=(2,2)),
    keras.layers.Conv2D(filters=256, 
                        kernel_size=(5,5), 
                        strides=(1,1), 
                        activation='relu', 
                        padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), 
                           strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), 
                        strides=(1,1), activation='relu', 
                        padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), 
                        strides=(1,1), activation='relu', 
                        padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), 
                        strides=(1,1), activation='relu', 
                        padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), 
                           strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])
# Uncomment for model summary
model.summary()

C:\Users\Amin\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 55, 55, 96)     │        11,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 55, 55, 96)     │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 27, 27, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 27, 27, 256)    │       614,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 27, 27, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 13, 13, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 13, 13, 384)    │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 13, 13, 384)    │         1,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 13, 13, 384)    │     1,327,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 13, 13, 384)    │         1,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 13, 13, 256)    │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 13, 13, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │    37,752,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │         4,097 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 58,267,713 (222.27 MB)

 Trainable params: 58,264,961 (222.26 MB)

 Non-trainable params: 2,752 (10.75 KB)

In [7]:
root_logdir = os.path.join(os.curdir, "my_logs")
def get_run_logdir():
    #TensorBosard run TITLE:
    run_id = ("AlexNet_Candidate_Model")
    return os.path.join(root_logdir,run_id)

run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

callbacks = [
    #comment next two lines to stop tensorboard if needed.
    keras.callbacks.ModelCheckpoint("C:/Users/Amin/Documents/cmmd/craig/Henry/ML_models/alexnet/save_at_{epoch}.keras"),
    keras.callbacks.TensorBoard(run_logdir)
]

In [8]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])
history = model.fit(train_generator, epochs=20, callbacks=callbacks, validation_data=validation_generator)

C:\Users\Amin\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20


C:\Users\Amin\AppData\Roaming\Python\Python312\site-packages\keras\src\backend\tensorflow\nn.py:750: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


301/301 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - binary_accuracy: 0.6037 - loss: 2.8143

C:\Users\Amin\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


301/301 ━━━━━━━━━━━━━━━━━━━━ 1451s 5s/step - binary_accuracy: 0.6036 - loss: 2.8139 - val_binary_accuracy: 0.2957 - val_loss: 0.9181
Epoch 2/20
301/301 ━━━━━━━━━━━━━━━━━━━━ 1429s 5s/step - binary_accuracy: 0.5995 - loss: 1.9038 - val_binary_accuracy: 0.7043 - val_loss: 0.6202
Epoch 3/20
301/301 ━━━━━━━━━━━━━━━━━━━━ 1427s 5s/step - binary_accuracy: 0.6142 - loss: 1.2136 - val_binary_accuracy: 0.7043 - val_loss: 0.6946
Epoch 4/20
301/301 ━━━━━━━━━━━━━━━━━━━━ 1465s 5s/step - binary_accuracy: 0.6378 - loss: 0.9328 - val_binary_accuracy: 0.7043 - val_loss: 0.6268
Epoch 5/20
301/301 ━━━━━━━━━━━━━━━━━━━━ 1430s 5s/step - binary_accuracy: 0.6230 - loss: 0.8476 - val_binary_accuracy: 0.7043 - val_loss: 0.6937
Epoch 6/20
301/301 ━━━━━━━━━━━━━━━━━━━━ 1430s 5s/step - binary_accuracy: 0.6119 - loss: 0.8123 - val_binary_accuracy: 0.7043 - val_loss: 0.6557
Epoch 7/20
301/301 ━━━━━━━━━━━━━━━━━━━━ 1427s 5s/step - binary_accuracy: 0.6174 - loss: 0.7762 - val_binary_accuracy: 0.7043 - val_loss: 0.8541
Epo

In [20]:
model.save("C:/Users/Amin/Documents/cmmd/alexnet_candidate_model.keras")